In [8]:
# Make sure to activate geopandas in anaconda prompt
# import libraries
import os
import folium
import geopandas as gpd
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [2]:
a = """
=========================================================================================================================
Where is your Next Eclipse in your Country?
This function asks you to input a name of county and outputs a year and cities that next eclipse passes over.
Then, an intereactive map will apper to see the cities and the eclipse path in your selected country.
Without further due, let's get started!
=========================================================================================================================
"""
print(a)


Where is your Next Eclipse in your Country?
This function asks you to input a name of county and outputs a year and cities that next eclipse passes over.
Then, an intereactive map will apper to see the cities and the eclipse path in your selected country.
Without further due, let's get started!



In [9]:
def NextEclipse():
    # Load in Data
    data_path = '../Data/'
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')   )
    ec = gpd.read_file(os.path.join(data_path,"TotEclip_2024_37.shp"))[['Name', 'FolderPath', 'Year','Date', 'geometry']]
    cities = gpd.read_file(os.path.join(data_path,"ne_10m_populated_places.shp"))

    # Edit data
    world.rename(columns={"name":"Country"}, inplace=True)
    cities = gpd.sjoin(cities, ec, how='inner', op='intersects')
    cities = cities.drop(['index_right'], axis=1)
    cities = gpd.sjoin(cities, world, how="inner", op="within")
    cities.rename(columns={"NAME":"City"}, inplace=True)    

    # Make a list of names of the countries within the exlipses.
    country_list2 = []
    for n in range(len(cities) ):
        cn=cities.iloc[n,11]
        if cn not in country_list2:
            country_list2.append(cn)
    country_list2.sort()
    
    # User input
    your_c = input("Type a name of country where you want to see your next eclipse. Type EXIT to end this program. ")
    map_stamen_water = "none"
    
    while  your_c != "EXIT":  
        if your_c in country_list2:
            print()
            print("Your input is in our database!")
            my_c= cities[cities.Country==your_c]
            my_country= world[(world.Country==your_c)]
            year = int(my_c[['Year']].min() )
            my_e= my_ec=ec[(ec['Year']== year)]
            my_c = my_c[(my_c.Year==year)] 

            print("Your next eclipse in your selected country, "+ str(your_c) + ", is in " + str(year) + "!" )
            print('Wow! There are {} cities in the eclipse path!'.format(len(my_c)))
            if len(my_c) >5:
                print("Here are the largest 5:")
                for n in range(5):
                    print(n+1,my_c.iloc[n,0] )
            else:
                print("Here are the largest"+str(len(my_c))+":")
                for n in range(len(my_c)):
                    print(n+1,my_c.iloc[n,0] )

            x = my_country.centroid.x.values[0]
            y = my_country.centroid.y.values[0]
            
            map_stamen_water = folium.Map()
            map_stamen_water = folium.Map(
                location=[y,x],
                tiles='Stamen Terrain',
                zoom_start=5
                )
            folium.GeoJson(my_c, name='Eclipse Cities', tooltip=folium.features.GeoJsonTooltip(fields=['Country', 'City'])).add_to(map_stamen_water)
            folium.GeoJson(my_e, name='Eclipse Path').add_to(map_stamen_water)
            print("Zoom in and hover the icons to see the city name.")
            display(map_stamen_water)

            your_c = input("If you wish to play this again, type a name of country name again, or type EXIT to reutrn your map and discontinue.")  
        else:
            print()
            print("Your input is not in our database. Here is the list of the country names for your reference.")
            print(country_list2)
            your_c = input("Type a name of country name again or type EXIT to return your map and discontinue. ")  

    
            
            
    print("""
    ************************************************************************************

    You are successfully ended the program! See you again:)

    ************************************************************************************
    """)    
    if map_stamen_water != "none":
        return map_stamen_water


In [10]:
myMap = NextEclipse()

Type a name of country where you want to see your next eclipse. Type EXIT to end this program. Japan

Your input is in our database!
Your next eclipse in your selected country, Japan, is in 2035!
Wow! There are 5 cities in the eclipse path!
Here are the largest5:
1 Toyama
2 Maebashi
3 Utsunomiya
4 Mito
5 Nagano
Zoom in and hover the icons to see the city name.


If you wish to play this again, type a name of country name again, or type EXIT to reutrn your map and discontinue.America

Your input is not in our database. Here is the list of the country names for your reference.
['Afghanistan', 'Algeria', 'Argentina', 'Australia', 'Botswana', 'Cameroon', 'Canada', 'Chad', 'China', 'Egypt', 'Finland', 'Greenland', 'Iceland', 'India', 'Iran', 'Japan', 'Kuwait', 'Lesotho', 'Libya', 'Mexico', 'Morocco', 'Namibia', 'New Zealand', 'Nigeria', 'North Korea', 'Norway', 'Pakistan', 'Russia', 'Saudi Arabia', 'Somalia', 'South Africa', 'Spain', 'Sudan', 'Sweden', 'Tunisia', 'United States of America', 'Yemen']
Type a name of country name again or type EXIT to return your map and discontinue. United States of America

Your input is in our database!
Your next eclipse in your selected country, United States of America, is in 2024!
Wow! There are 46 cities in the eclipse path!
Here are the largest 5:
1 Paragould
2 Cleburne
3 Arlington
4 Montpelier
5 Cape Girardea

If you wish to play this again, type a name of country name again, or type EXIT to reutrn your map and discontinue.EXIT

    ************************************************************************************

    You are successfully ended the program! See you again:)

    ************************************************************************************
    


In [11]:
display(myMap)

In [12]:
# Save the results as a HTML file for easy sharing
myMap.save('mymap.html')